In [1]:
# !pipenv install pyodbc

In [2]:
import pandas as pd
import numpy as np
import base64
import pyodbc
import os

In [3]:
# Create Connection
password = config.password
# Connecting to SQL Server
client_code = '544'
database = 'SS'
base_dir = r"C:\Users\jesse.henson\StaffScapes\StaffScapes - Documents\IT\Isolved Import\prior_balance\imports"
conn = pyodbc.connect('Driver={SQL Server Native Client 10.0};'
                      'Server=staffc-darwin\darwin12;'
                      f'Database={database};'
                      'UID=sa;'
                      'PWD=' + password + ';')
if not os.path.isdir(f'{base_dir}/{client_code}'):
    os.mkdir(f'{base_dir}/{client_code}')

In [4]:
# Paycode Data
# Read it 
sql_str = f"Select EMPLOYID as [Key], DEPRTMNT as LaborValue2, CHEKNMBR as CheckNumber, AUCTRLCD as ControlNumber,  CHEKDATE as CheckDate,  UNTSTOPY as hours, PYRLRTYP as PayType, PAYROLCD as PayCode, uprtrxam as dollars, TRXBEGDT as PeriodBeginDate, TRXENDDT as PeriodEndDate from upr30300 where employid between '{'F1' if database == 'AHR' else '01'}{client_code}0001' and '{'F1' if database == 'AHR' else '01'}{client_code}9999'"
paycode_data = pd.read_sql_query(sql_str,conn,parse_dates=['PeriodBeginDate','PeriodEndDate','CheckDate'])
paycode_data['PayPrefix'] = paycode_data['PayType'].map({1:'E_', 4:'T_', 2:'D_', 3:'M_'})
paycode_data['PayCode'] = paycode_data['PayCode'].apply(lambda x:x.strip())
paycode_data['PayCode'].replace(to_replace='BONUS~',value='BONUS',inplace=True)
paycode_data['PayCode'] = paycode_data['PayPrefix'] + paycode_data['PayCode']
paycode_data['PayCode'] = paycode_data['PayCode'].fillna('Missing')
paycode_data['PayCodeHour'] = paycode_data['PayCode'].apply(lambda x: x + '_Hours' if x[0] == 'E' else x)
paycode_data['PayCode'] = paycode_data['PayCode'].apply(lambda x: x + '_Dollars' if x[0] in ['M','E'] else x)
# paycode_data['PayCode'].unique()

In [5]:
ben_ded_bp_dict = {}

def closest_word(written_word, target_word):
    score = 0
    number_of_matches = 0
    temp_written = written_word
    temp_target = target_word
    for written_char in written_word:
        if written_char in target_word:
            target_word = target_word.replace(written_char, '',1)
            written_word = written_word.replace(written_char, '',1)
            number_of_matches += 1
            score += 100
        for char_left in written_word: 
            if char_left in temp_target:
                score += 50
            else: 
                score -= 10
        for char_left in target_word:
            if char_left in temp_written:
                score += 50
            else:
                score -= 10
    return (score)

    



In [6]:
ded_score_keeper = []



bp_ded_codes = paycode_data[paycode_data['PayCode'].str.count('D_BP') > 0].PayCode
bp_ded_arr = bp_ded_codes.unique()

ded_codes = paycode_data[paycode_data['PayCode'].str.count('^D_((?!BP).)*$') > 0].PayCode
ded_arr = ded_codes.unique()
ded_arr

for bp_ded in bp_ded_arr:
    ded_score_keeper = []
    for ded in ded_arr:
        temp_score = closest_word(bp_ded.strip()[bp_ded.index('-')+1:], ded.strip()[ded.index('_')+1:])
        ded_score_keeper.append((temp_score, bp_ded, ded))
    ben_ded_bp_dict[max(ded_score_keeper)[1]] = max(ded_score_keeper)[2]



In [7]:
ben_score_keeper = []



bp_ben_codes = paycode_data[paycode_data['PayCode'].str.count('M_BP') > 0].PayCode
bp_ben_arr = bp_ben_codes.unique()

ben_codes = paycode_data[paycode_data['PayCode'].str.count('M_((?!BP).)*$') > 0].PayCode
ben_arr = ben_codes.unique()
ben_arr

for bp_ben in bp_ben_arr:
    ben_score_keeper = []
    for ben in ben_arr:
        temp_score = closest_word(bp_ben.strip()[bp_ben.index('-')+1:bp_ben.index('Dollars')], ben.strip()[ben.index('_')+1:ben.index('Dollars')])
        ben_score_keeper.append((temp_score, bp_ben, ben))
    ben_ded_bp_dict[max(ben_score_keeper)[1]] = max(ben_score_keeper)[2]

# print(bp_ben_arr)
# print(ben_arr)
print(ben_ded_bp_dict)
# ben_ded_bp_dict['M_BP-VIS_Dollars'] = 'M_VIS_Dollars'
print(ben_ded_bp_dict)
# ben_ded_bp_dict['D_BP-MD1'] = 'D_UHC-1'

{'D_BP-DN1': 'D_DNT-1', 'D_BP-VS1': 'D_VIS-1', 'D_BP-MD1': 'D_MED-1', 'D_BP-MDT': 'D_DNT-T', 'D_BP-ACI': 'D_AC-CI', 'M_BP-DNT_Dollars': 'M_DNT_Dollars', 'M_BP-VIS_Dollars': 'M_VIS_Dollars', 'M_BP-MED_Dollars': 'M_MED_Dollars', 'M_BP-UHC_Dollars': 'M_UHC_Dollars'}
{'D_BP-DN1': 'D_DNT-1', 'D_BP-VS1': 'D_VIS-1', 'D_BP-MD1': 'D_MED-1', 'D_BP-MDT': 'D_DNT-T', 'D_BP-ACI': 'D_AC-CI', 'M_BP-DNT_Dollars': 'M_DNT_Dollars', 'M_BP-VIS_Dollars': 'M_VIS_Dollars', 'M_BP-MED_Dollars': 'M_MED_Dollars', 'M_BP-UHC_Dollars': 'M_UHC_Dollars'}


In [8]:
paycode_pivot_dollars = paycode_data.copy()
paycode_pivot_dollars = paycode_pivot_dollars.pivot_table(index=['Key','ControlNumber'], columns='PayCode', fill_value=0, values=['dollars'], aggfunc=np.sum)
paycode_pivot_dollars.columns = paycode_pivot_dollars.columns.droplevel(level=0)
paycode_pivot_dollars.reset_index(inplace=True)
# paycode_pivot_dollars[paycode_pivot_dollars['Key'] == '015400051      ']['E_HOURLY_Dollars']
# paycode_pivot_dollars.filter(like='015400051', axis=0)

In [9]:
paycode_pivot_hours = paycode_data.copy()
paycode_pivot_hours = paycode_pivot_hours.pivot_table(index=['Key','ControlNumber'], columns='PayCodeHour', fill_value=0, values=['hours'], aggfunc=np.sum)
paycode_pivot_hours.columns = paycode_pivot_hours.columns.droplevel(level=0)
paycode_pivot_hours.reset_index(inplace=True)
columns = [col for col in paycode_pivot_hours.columns if 'Hours' not in col and col != 'Key' and col != 'ControlNumber']
paycode_pivot_hours = paycode_pivot_hours[paycode_pivot_hours.columns.drop(columns)]
# paycode_pivot_hours

In [10]:
paycode_pivot = paycode_pivot_hours.merge(paycode_pivot_dollars, on=['Key','ControlNumber'])
# paycode_pivot

In [11]:
# Create non-pivoted table
check_meta_data = paycode_data.loc[:,['Key','CheckNumber','CheckDate','LaborValue2','PeriodBeginDate','PeriodEndDate', 'ControlNumber']]
check_meta_data = check_meta_data.drop_duplicates(subset=['Key','ControlNumber'])
# check_meta_data

In [12]:
# Check data
sql_str = f"Select EMPLOYID as [Key],AUCTRLCD as ControlNumber, FDWDGPRN + FDTXTIPS as T_FEDWT, FICAMWPR + FICMTPTX as T_MEDEE, FCASWPR + FICSTPTX as T_SOCSECEE from upr30100 where employid between '{'F1' if database == 'AHR' else '01'}{client_code}0001' and '{'F1' if database == 'AHR' else '01'}{client_code}9999'"
check_data = pd.read_sql_query(sql_str,conn)
print(check_data.columns)
# check_data

Index(['Key', 'ControlNumber', 'T_FEDWT', 'T_MEDEE', 'T_SOCSECEE'], dtype='object')


In [13]:
# FUTA SUTA Data
sql_str = f"select employid as [Key], AUCTRLCD as ControlNumber, PYRLRTYP as payType, TW_FUTA_Liability, TXBLWAGS as #TaxableWage from tw80507 where employid between '{'F1' if database == 'AHR' else '01'}{client_code}0001' and '{'F1' if database == 'AHR' else '01'}{client_code}9999'"
employer_data = pd.read_sql_query(sql_str,conn)
# Pivot data
employer_data[['Key','ControlNumber','payType','TW_FUTA_Liability']]
employer_data_pivot = employer_data[['Key','ControlNumber','payType','TW_FUTA_Liability']].pivot_table(index=['Key','ControlNumber'], columns='payType')
new_columns = ['T_SUIER','# FUTA From Darwin','T_WRKCOMPER']
employer_data_pivot.columns = new_columns
employer_data_pivot

employer_data_pivot.reset_index(inplace=True)
# employer_data_pivot

In [14]:


# Creat Table for taxable wage
taxable_wage = employer_data[['Key','ControlNumber','#TaxableWage','payType']].query('payType == 2')
# taxable_wage


In [15]:
# Merge the two tables
employer_data_final = pd.merge(left=employer_data_pivot, right=taxable_wage, on=['Key','ControlNumber'])

# Create FUTA liability actual
employer_data_final['T_FUTAER'] = employer_data_final['#TaxableWage'] * .006


# employer_data_final
# employer_data

In [16]:
# Merge all three 

In [17]:
prior_history_data = pd.merge(left=employer_data_final,right=check_data,on=['Key','ControlNumber'])
prior_history_data = pd.merge(left=prior_history_data, right=check_meta_data, on=['Key','ControlNumber'])
prior_history_data = pd.merge(left=prior_history_data, right=paycode_pivot, on=['Key','ControlNumber'])
prior_history_data.rename(columns = {"ControlNumber": "#ControlNumber"}, inplace=True)
prior_history_data['Key'] = prior_history_data['Key'].str[2:]




prior_history_data['E_REG_Hours'] = prior_history_data['E_SALARY_Hours'] + prior_history_data['E_HOURLY_Hours']
prior_history_data['E_REG_Dollars'] = prior_history_data['E_SALARY_Dollars'] + prior_history_data['E_HOURLY_Dollars']
prior_history_data = prior_history_data.drop(columns=['E_SALARY_Hours','E_SALARY_Dollars','E_HOURLY_Hours','E_HOURLY_Dollars'])

# prior_history_data.rename(columns={'E_HOURLY_Hours':'E_REG_Hours','E_HOURLY_Dollars':'E_REG_Dollars'},inplace=True)
# prior_history_data.rename(columns={'E_SALARY_Hours':'E_REG_Hours','E_SALARY_Dollars':'E_REG_Dollars'}, inplace=True)


try: 
    prior_history_data.rename(columns={prior_history_data.filter(regex='D_TAX.*').columns[0]: 'T_HEAD',
                                      prior_history_data.filter(regex='M_TAX.*').columns[0]: 'T_LOCMISCER'}, inplace = True)
except:
    print('no head tax')
prior_history_data.rename(columns={'T_CO':'T_STATEWT', 'payType':'#payType','D_CHILD1':'G_Child Support_1','G_GRN1':'G_Child Support_1'}, inplace=True)

print(ben_ded_bp_dict)


for bp_code, reg_code in ben_ded_bp_dict.items():
    prior_history_data['#' + reg_code] = prior_history_data[reg_code] + prior_history_data[bp_code]
    prior_history_data[reg_code] = prior_history_data['#' +reg_code]  

    
columns = [key for key, value in ben_ded_bp_dict.items()]
          
prior_history_data = prior_history_data.drop(columns=columns)
prior_history_data = prior_history_data.loc[:, (prior_history_data != 0).any(axis=0)]
prior_history_data = prior_history_data.round(2)
prior_history_data = prior_history_data[prior_history_data['CheckDate']>'01/01/2021']
# prior_history_data['E_PTO_Dollars'] = prior_history_data['E_PTO_Dollars'] + prior_history_data['E_PTO-M_Dollars']
# prior_history_data['E_PTO_Hours'] = prior_history_data['E_PTO_Hours'] + prior_history_data['E_PTO-M_Hours']
# prior_history_data.rename(columns={'E_PTO-M_Hours': '#E_PTO-M_Hours','E_PTO-M_Dollars':'#E_PTO-M_Dollars'}, inplace=True)
prior_history_data.drop_duplicates(subset=['Key','#ControlNumber'], inplace=True)
prior_history_data.to_csv(f"{base_dir}/{client_code}/{client_code}_prior_balance.csv")
prior_history_data

{'D_BP-DN1': 'D_DNT-1', 'D_BP-VS1': 'D_VIS-1', 'D_BP-MD1': 'D_MED-1', 'D_BP-MDT': 'D_DNT-T', 'D_BP-ACI': 'D_AC-CI', 'M_BP-DNT_Dollars': 'M_DNT_Dollars', 'M_BP-VIS_Dollars': 'M_VIS_Dollars', 'M_BP-MED_Dollars': 'M_MED_Dollars', 'M_BP-UHC_Dollars': 'M_UHC_Dollars'}


,Key,#ControlNumber,T_SUIER,# FUTA From Darwin,T_WRKCOMPER,#TaxableWage,#payType,T_FUTAER,T_FEDWT,T_MEDEE,...,E_REG_Dollars,#D_DNT-1,#D_VIS-1,#D_MED-1,#D_DNT-T,#D_AC-CI,#M_DNT_Dollars,#M_VIS_Dollars,#M_MED_Dollars,#M_UHC_Dollars
224,5440008,UPRCC00049362,58.19,22.49,8.17,2811.35,2,16.87,421.55,40.76,...,2519.06,16.3,3.41,169.25,0.0,0.0,0.0,0.0,50.0,0
225,5440008,UPRCC00049462,59.14,22.85,8.30,2856.79,2,17.14,431.55,41.43,...,2647.26,16.3,3.40,169.25,0.0,0.0,0.0,0.0,50.0,0
226,5440008,UPRCC00049574,58.38,10.65,8.20,1331.86,2,7.99,423.50,40.89,...,3009.16,16.3,3.41,169.25,0.0,0.0,0.0,0.0,50.0,0
227,5440008,UPRCC00049691,56.26,0.00,7.92,0.00,2,0.00,400.97,39.41,...,2810.50,16.3,3.40,169.25,0.0,0.0,0.0,0.0,50.0,0
228,5440008,UPRCC00049788,49.55,0.00,8.08,0.00,2,0.00,413.51,40.23,...,2771.23,16.3,3.41,169.25,0.0,0.0,0.0,0.0,50.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2353,5440099,UPRCC00050230,34.87,13.48,4.59,1684.53,2,10.11,0.00,24.43,...,1684.53,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0
2354,5440099,UPRCC00050298,28.29,10.93,3.72,1366.54,2,8.20,0.00,19.81,...,1366.54,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0
2355,5440099,UPRVC00002640,-27.18,-10.51,-3.58,-1313.13,2,-7.88,0.00,-19.04,...,-1313.13,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0
2356,5440100,UPRCC00050179,15.32,5.92,2.02,740.00,2,4.44,0.00,10.73,...,740.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0


In [18]:
'''
TODO
Combine salary and hourly into Reg - add together the two columns and rename to E_REG_Dollars/hours
get rid of additional D/T/m columns
Delete Bonus/Comm/Flat column Hours
Set M in front when Memo 
Set T_<<<STATE>>> to T_STATEWT
'''

'\nTODO\nCombine salary and hourly into Reg - add together the two columns and rename to E_REG_Dollars/hours\nget rid of additional D/T/m columns\nDelete Bonus/Comm/Flat column Hours\nSet M in front when Memo \nSet T_<<<STATE>>> to T_STATEWT\n'

In [19]:
flag = False

for column in prior_history_data.columns:
    if 'BP' in column:
        flag = True
        
flag

False